In [ ]:
def download(item):
    return item

def resize(item):
    return item

def upload(item):
    return item

from collections import deque
from threading import Lock

class MyQueue:
    def __init__(self):
        self.items = deque()
        self.lock = Lock()

 # 생산자(producer)인 디지털 카메라는 새 이미지를 대기 아이템 리스트의 끝에 추가
    def put(self, item):
        with self.lock:
            self.items.append(item)

# 소비자(consumer)인 처리 파이프라인의 첫 번쨰 단계에서는 대기 아이템 리스트의 앞쪽에서 이미지 추출
    def get(self):
        with self.lock:
            return self.items.popleft()


# 여기서는 이러한 큐에서 작업을 꺼내와서 함수를 실행한 후 결과를 또 다른 큐에 넣는 파이썬 스레드로 파이프라인의 각 단계를 표현
# 또한 작업 스레드가 새 입력을 몇 번이나 체크하고 작업을 얼마나 완료하는지 추적

from threading import Thread
import time

class Worker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super().__init__()
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.polled_count = 0
        self.work_done = 0

    # 가장 까다로운 부분은 이전 단계에서 아직 작업을 완료하지 않아 입력 큐가 비어 있는 경우를 작업스레드에서 적절히 처리하는 것
    # 다음 코드에서 IndexError 예외를 잡는 부분이 이에 해당. 이 경우를 조립 라인이 정체된 상황이라고 보면 됨

    def run(self):
        while True:
            self.polled_count += 1
            try:
                item = self.in_queue.get()
            except IndexError:
                time.sleep(0.01) # 할 일이 없음
            else:
                result = self.func(item)
                self.out_queue.put(result)
                self.work_done += 1

# 이제 작업 조율용 큐와 그에 해당하는 작업 스레드를 생성해 세 단계를 연결하면 됨

download_queue = MyQueue()
resize_queue = MyQueue()
upload_queue = MyQueue()
done_queue = MyQueue()

threads = [
    Worker(download, download_queue, resize_queue),
    Worker(resize, resize_queue, upload_queue),
    Worker(upload, upload_queue, done_queue),
]

# 스레드를 시작하고 파이프라인의 첫 번째 단계에 많은 작업을 추가. 다음 코드에서 download 함수에 실제 데이터 대신 일반 object 인스턴스를 사용


for thread in threads:
    thread.start()

for _ in range(1000):
    download_queue.put(object())

while len(done_queue.items) < 1000:
    # 기다리는 동안 유용한 작업을 수행한다
    pass


# 이 코드는 제대로 동작하지만, 입력 큐에서 새 작업을 가져오는 스레드가 일으키는 흥미로운 부작용이 존재.
# 바로 run 메서드에서 IndexError 예외를 잡는 그 까다로운 부분이 매우 많이 실행됨


processed = len(done_queue.items)
polled = sum(t.polled_count for t in threads)
print(f'{processed} 개의 아이템을 처리했습니다, '
      f'이때 폴링을 {polled} 번 했습니다.')

# 작업이 끝나도 무한대기함. 프로그램을 강제종료시킬것